In [ ]:
%load_ext autoreload  
%autoreload 2 
%matplotlib inline

In [2]:
from datasets import Dataset
import torch
from torch import nn
from torch.utils.data import DataLoader
from transformer import Transformer

In [3]:
def get_label(x_data, y_data, x_idx, y_idx, p, operation):
    x, y = x_data[x_idx], y_data[y_idx]
    return operation(x, y, p) 

def div_mod(x, y, p):
    res = (x/y) % p
    return res

In [4]:
def test_div_mod():
    expected_results = [
        (0, 5, 0),
        (5, 5, 1),
        (90, 3, 30),
        (90, 2, 45)
    ]
    for x, y, result in expected_results:
        assert div_mod(x, y, 97) == result

test_div_mod()

In [5]:
def create_data(x_data, y_data, p, operation):
    """
    Returns: 
        `data` flattened 1D array of size len(x_data) * len(y_data)
    """
    data = {"operand": [], "label": []}
    for i in range(len(x_data)):
        for j in range(len(y_data)):
            data["operand"].append((x_data[i], y_data[j]))
            data["label"].append(get_label(x_data, y_data, i, j, p, operation))
    return data

In [6]:
def test_create_data():
    p = 97
    x_data = list(range(p))
    y_data = list(range(1, p))
    data = create_data(x_data, y_data, p, div_mod)
    assert len(data["operand"]) == len(data["label"]) == len(x_data) * len(y_data)

    expected_results = [(-1, (96,96), 1),
                        (0, (0, 1), 0),
                        (96, (1, 1), 1),
                        (192, (2, 1), 2)]

    for position, operand, label in expected_results:
        assert data["operand"][position] == operand
        assert data["label"][position] == label

test_create_data()

In [7]:
p = 97
x_data = list(range(p))
y_data = list(range(1, p))

In [8]:
data = create_data(x_data, y_data, p, div_mod)
ds = Dataset.from_dict(data)
ds.set_format(type='torch', columns=['operand', 'label'])

In [9]:
train_dl = DataLoader(ds, batch_size=8, shuffle=False)

In [10]:
for x in train_dl:
    print(x['operand'], x['operand'].dtype)
    print(x['label'], x['label'].dtype)
    break

tensor([[0, 1],
        [0, 2],
        [0, 3],
        [0, 4],
        [0, 5],
        [0, 6],
        [0, 7],
        [0, 8]]) torch.int64
tensor([0., 0., 0., 0., 0., 0., 0., 0.]) torch.float32


In [11]:
model = Transformer(n_layers=2, n_heads=4, d_model= 128)

In [12]:
next(iter(train_dl))["operand"]

tensor([[0, 1],
        [0, 2],
        [0, 3],
        [0, 4],
        [0, 5],
        [0, 6],
        [0, 7],
        [0, 8]])

In [13]:
model(next(iter(train_dl))["operand"])

x: tensor([[0, 1],
        [0, 2],
        [0, 3],
        [0, 4],
        [0, 5],
        [0, 6],
        [0, 7],
        [0, 8]])


RuntimeError: expected scalar type Double but found Float